<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Examples.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Examples: Clustering and geospatial analysis
© ExploreAI Academy

In this train, we'll use geopandas to plot geospatial data and create multidimensional plots by adjusting size and colour.

## Learning objectives

By the end of this train, you should be able to;
- Create multidimensional plots by adjusting size and colour;
- Use geopandas to plot geospatial data.

## Introduction

In this train we will introduce multi-dimensional plotting and work with geospatial data. Let's start by importing the packages we'll need for this train.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
%matplotlib inline

Before you continue, make sure you install this library using `conda install geopandas`. If you are struggling to correctly install the `geopandas` package, run this notebook on Google Colab and install the package using the following cell:

In [ ]:
#!pip install geopandas

## Preparing the Data
In this train, we will be using the same preprocessing and clustering technique as in the Hierarchical Clustering train, but just adjusted for use with United States census data. Let's load the data and take a look at it.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/unsupervised_sprint/acs2015_county_data.csv', encoding='utf-8').dropna()
df.head()

We'll set the labels to `CensusID`, `State`, and `County`, and scale the features using `StandardScaler`. 

In [ ]:
labels = ['CensusId', 'State', 'County']
features = [col for col in df.columns if col not in labels]

scaler = StandardScaler()
df[features] = df[features].astype('float64')
X = scaler.fit_transform(df[features])

We'll now cluster our data using `AgglomerativeClustering`, with 4 clusters. In this train, we will be using [Ward's minimum variance method](https://en.wikipedia.org/wiki/Ward%27s_method) for our linkage method with the Euclidean distance metric. You are encouraged to try out different distance metrics and linkage methods and see how your results differ.   

In [ ]:
hc = AgglomerativeClustering(n_clusters=4, linkage='ward', affinity='euclidean').fit(X)
df['cluster'] = hc.labels_ + 1
df['cluster'] = df['cluster'].astype('int64')

## Multi-dimensional plotting
Before we plot the geospatial data, let's first look at another way to fit more features into a single plot. In the Hierarchical Clustering train, we only plotted data using the horizontal and vertical axes, but we never looked at the different visual channels.  

Let's use both shape and colour to add two new dimensions to our plot. First, we will decide on a suitable palette with four colours:

In [ ]:
sns.palplot(sns.color_palette("Set1", n_colors=4))
sns.palplot(sns.color_palette("Set2", n_colors=4))
sns.palplot(sns.color_palette("Set3", n_colors=4))
sns.palplot(sns.color_palette("Paired", n_colors=4))

#### Custom colour palettes
Seaborn also provides support for using names from this [xkcd crowdsourced effort](https://blog.xkcd.com/2010/05/03/color-survey-results/) to name random RGB colours. You can find the list of colours [here](https://xkcd.com/color/rgb/).

In [ ]:
# choose four colours
colours = ["windows blue", "amber", "coral", "faded green"]
# plot them as a palette
sns.palplot(sns.xkcd_palette(colours))

Let's now use this colour palette to help us visualise our clusters while plotting `IncomePerCap` vs `MeanCommute`. We'll also adjust the size of the points according to the rate of `Unemployment`.

In [ ]:
# create our colour palette
cmap = sns.xkcd_palette(colours)
plt.figure(figsize=(12,8))
ax = sns.scatterplot(x="IncomePerCap", y="MeanCommute",
                     hue="cluster", size="Unemployment",
                     palette=cmap, sizes=(10, 200),
                     data=df)
plt.show()

This is just a nudge in the right direction. You are encouraged to do some more interrogations of the clusters to gain further insights.

> Make use of the [seaborn example gallery](https://seaborn.pydata.org/examples/index.html) for some inspiration

## Quick introduction to geospatial data analysis
We will now work through an introduction to a really cool Python library for working with geospatial data, called [GeoPandas](http://geopandas.org/). A lot more can be done with this package than what we do here. We encourage you to explore this package more.

To load a GeoDataFrame, you do it in a very similar way to loading a regular Pandas DataFrame. We have provided a [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) called `us-counties.json` which can be read straight into GeoPandas a GeoDataFrame:

In [ ]:
gdf = gpd.read_file('https://raw.githubusercontent.com/python-visualization/folium/master/tests/us-counties.json')

GeoDataFrames inherit all the same methods as regular DataFrames. The only difference is that a GeoDataFrame must have a column named `geometry` which contains the spatial features relating to each observation in the data.   

There are three basic classes of geometric objects:   
- Points (x,y coordinate pairs);
- Lines (multiple points which define segments); and
- Polygons (multiple connected lines which form an enclosed shape).

In [ ]:
gdf.head()

Our GeoDataFrame consists of all the US counties and their respective polygons. Let's merge this GeoDataFrame with our original DataFrame:

In [ ]:
gdf['id'] = gdf['id'].astype('int64')
df = pd.merge(left=df, right=gdf, how='inner', left_on='CensusId', right_on='id')
del df['id']
del df['name']
# convert to a GeoDataFrame, specifying which column we wish to use for the geometry info
df = gpd.GeoDataFrame(df, geometry='geometry')
df.head(3)

Now, we can plot the counties as a map and adjust the visuals according to the rate of `Unemployment`:

In [ ]:
df.plot(column='Unemployment',
        cmap='Reds',
        figsize=(18,16))
plt.show()

Let's plot the clusters on the map and see where in the United States these clusters are:

In [ ]:
df.plot(column='cluster',
        cmap='tab20c',
        figsize=(18,16),
        categorical=True,
        legend=True)
plt.show()

## Conclusion

In this train, we clustered United States census data, plotted this data multidimensionally, and used `geopandas` to plot geospatial data on a map. This train was a brief introduction to these topics. The reader is encouraged to explore them further.

## Additional Links
[The Art of Effective Visualisation of Multi-dimensional Data](https://towardsdatascience.com/the-art-of-effective-visualization-of-multi-dimensional-data-6c7202990c57)

[Some `geopandas` examples](https://geopandas.readthedocs.io/en/latest/gallery/)

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>